# <center>Regressão Linear - Previsão de IMC

 Legendas:
   * <code style="color:green">Teoria/Instruções</code>
   * <code style="color:purple">Comentários</code>
   * <code style="color:red">Dúvidas a serem esclarecidas</code>

### <center>Importando Bibliotecas

In [ ]:
import pandas as pd # importando a biblioteca de manipulação de dados
import statsmodels.api as sm # biblioteca para a regressão logística
import numpy as np # biblioteca para calculo
from matplotlib import pyplot as plt # importando a biblioteca de visualização de dados 
import seaborn as sns# importando a biblioteca de visualização de dados 
from tabulate import tabulate # biblioteca para estilizar tabelas
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score # funções de erro
from statsmodels.stats.outliers_influence import variance_inflation_factor # funções para calcular o vif
import scipy.stats as stats # biblioteca para modelagem
import pylab # biblioteca para o qqplot
from statsmodels.stats.diagnostic import het_breuschpagan # função para o teste de breusch-pagan
from scipy.stats import shapiro # teste de normalidade
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor

### <center>Funções para seleção de variáveis utilizando métodos backward, forward e stepwise:

In [ ]:
# Esta função realiza uma seleção forward stepwise com base no p-valor das variáveis independentes.
# A cada passo, adiciona a variável independente com o menor p-valor ao modelo, desde que o p-valor 
# seja menor que o nível de significância especificado.
# Parâmetros:
# - var_dependente: Nome da variável dependente.
# - var_independente: Lista de variáveis independentes a serem avaliadas.
# - base: Conjunto de dados contendo as variáveis dependentes e independentes.
# - signif: Nível de significância para a inclusão das variáveis (por exemplo, 0.05).
# Retorna: DataFrame contendo as variáveis selecionadas e seus respectivos p-valores.
# criada por Mateus Rocha - time ASN.Rocks

def selecionar_pvalor_forward(var_dependente, var_independente, base, signif):

    preditoras = []
    pvalor_preditoras = []
    Y = base[var_dependente]
    while True and var_independente != [] :
        lista_pvalor = []
        lista_variavel = []
        for var in var_independente:
            X = sm.add_constant(base[ [var] +  preditoras ])
            
            modelo = sm.OLS(Y,X).fit()
            
            if( preditoras == []):
    
                pvalor = modelo.pvalues[1]
                variavel = modelo.pvalues.index[1]
            
            else:
                pvalor = modelo.pvalues.drop(preditoras)[1]
                variavel = modelo.pvalues.drop(preditoras).index[1]
                
            lista_pvalor.append(pvalor)
            lista_variavel.append(variavel)          
        
        if( lista_pvalor[ np.argmin(lista_pvalor) ] < signif ):
            preditoras.append( lista_variavel[np.argmin(lista_pvalor)] )
            pvalor_preditoras.append(lista_pvalor[ np.argmin(lista_pvalor) ])
            var_independente.remove( lista_variavel[ np.argmin(lista_pvalor)] )
        else:
            break
    info_final = pd.DataFrame({ 'var': preditoras, 'pvalor': pvalor_preditoras})
    return info_final

# Exemplo de uso
# colunas_pvalor = selecionar_pvalor_forward(var_dependente='bodyfat', var_independente=X.columns.to_list(), base=Bodyfat, signif=0.05)
# colunas_pvalor

In [ ]:
# Esta função realiza uma seleção forward stepwise com base no critério de informação de Akaike (AIC).
# A cada passo, adiciona a variável independente que minimiza o AIC ao modelo.
# Parâmetros:
# - var_dependente: Nome da variável dependente.
# - var_independente: Lista de variáveis independentes a serem avaliadas.
# - base: Conjunto de dados contendo as variáveis dependentes e independentes.
# Retorna: DataFrame contendo as combinações de variáveis selecionadas e seus respectivos AICs, 
# ordenados do menor para o maior AIC.
# criada por Mateus Rocha - time ASN.Rocks

def selecionar_aic_forward(var_dependente, var_independente, base):
    
    preditoras = []
    aic_preditoras = []
    Y = base[var_dependente]
    lista_final = []
    aic_melhor = float('inf')
    
    while True and var_independente != []:
        lista_aic = []
        lista_variavel = []
        lista_modelos =[]
        if(var_independente == []):
            break
        for var in var_independente:
            X = sm.add_constant(base[ [var] +  preditoras ])
            aic = sm.OLS(Y,X).fit().aic
            variavel = var
                
            lista_aic.append(aic)
            
            lista_variavel.append(var)
            
            lista_modelos.append( [var] +  preditoras )
            
        if( lista_aic[ np.argmin(lista_aic) ] < aic_melhor ):
            
            lista_final.append(lista_modelos[ np.argmin(lista_aic)]  )
            
            preditoras.append( lista_variavel[np.argmin(lista_aic)] )
            
            aic_preditoras.append(lista_aic[ np.argmin(lista_aic) ])
            
            var_independente.remove( lista_variavel[ np.argmin(lista_aic)] )
            
            aic_melhor = lista_aic[ np.argmin(lista_aic) ] 
            
        else:
            break
        
    info_final = pd.DataFrame({ 'var': lista_final, 'aic': aic_preditoras}).sort_values(by = 'aic')
    return info_final

# Exemplo de uso
# colunas_aic = selecionar_aic_forward(var_dependente='bodyfat', var_independente=X.columns.to_list(), base=Bodyfat)
# colunas_aic

In [ ]:
# Esta função realiza uma seleção forward stepwise com base no critério de informação bayesiano (BIC).
# A cada passo, adiciona a variável independente que minimiza o BIC ao modelo.
# Parâmetros:
# - var_dependente: Nome da variável dependente.
# - var_independente: Lista de variáveis independentes a serem avaliadas.
# - base: Conjunto de dados contendo as variáveis dependentes e independentes.
# Retorna: DataFrame contendo as combinações de variáveis selecionadas e seus respectivos BICs, 
# ordenados do menor para o maior BIC.
# criada por Mateus Rocha - time ASN.Rocks

def selecionar_bic_forward(var_dependente, var_independente, base):
    
    preditoras = []
    bic_preditoras = []
    Y = base[var_dependente]
    lista_final = []
    bic_melhor = float('inf')
    
    while True and var_independente != []:
        lista_bic = []
        lista_variavel = []
        lista_modelos =[]
        if(var_independente == []):
            break
        for var in var_independente:
            X = sm.add_constant(base[ [var] +  preditoras ])
            bic = sm.OLS(Y,X).fit().bic
            variavel = var
                
            lista_bic.append(bic)
            
            lista_variavel.append(var)
            
            lista_modelos.append( [var] +  preditoras )
            
        if( lista_bic[ np.argmin(lista_bic) ] < bic_melhor ):
            
            lista_final.append(lista_modelos[ np.argmin(lista_bic)]  )
            
            preditoras.append( lista_variavel[np.argmin(lista_bic)] )
            
            bic_preditoras.append(lista_bic[ np.argmin(lista_bic) ])
            
            var_independente.remove( lista_variavel[ np.argmin(lista_bic)] )
            
            aic_melhor = lista_bic[ np.argmin(lista_bic) ] 
            
        else:
            break
        
    info_final = pd.DataFrame({ 'var': lista_final, 'bic': bic_preditoras}).sort_values(by = 'bic')
    return info_final

# Exemplo de uso
# colunas_bic = selecionar_bic_forward(var_dependente='bodyfat', var_independente=X.columns.to_list(), base=Bodyfat)
# colunas_bic

In [ ]:
# Esta função realiza uma seleção backward stepwise com base no p-valor das variáveis independentes.
# A cada passo, remove a variável independente com o maior p-valor do modelo, 
# desde que seja maior que o nível de significância especificado.
# Parâmetros:
# - var_dependente: Nome da variável dependente.
# - var_independente: Lista de variáveis independentes a serem avaliadas.
# - base: Conjunto de dados contendo as variáveis dependentes e independentes.
# - signif: Nível de significância para a remoção das variáveis (por exemplo, 0.05).
# Retorna: DataFrame contendo as variáveis restantes após a seleção backward.
# criada por Mateus Rocha - time ASN.Rocks

def selecionar_pvalor_backward(var_dependente, var_independente, base, signif):
    Y = base[var_dependente]
    
    while True and var_independente != []:
        
        X_geral = sm.add_constant(base[var_independente])
        
        modelo = sm.OLS(Y,X_geral).fit()
        
        pvalor_geral = modelo.pvalues
        
        variavel_geral = modelo.pvalues.index
        
        if(pvalor_geral[ np.argmax(pvalor_geral) ] > signif ):
            var_independente.remove( variavel_geral[ np.argmax(pvalor_geral) ] )
        else:
            break
    
    
    
    info_final = pd.DataFrame({ 'var': var_independente})
    return info_final

# Exemplo de uso
# colunas_pvalor_back = selecionar_pvalor_backward(var_dependente='bodyfat', var_independente=X.columns.to_list(), base=Bodyfat, signif=0.05)
# colunas_pvalor_back

In [ ]:
# Esta função realiza uma seleção backward stepwise com base no critério de informação de Akaike (AIC).
# A cada passo, remove a variável que minimiza o AIC, desde que a remoção resulte em um AIC menor do que o modelo atual.
# Parâmetros:
# - var_dependente: Nome da variável dependente.
# - var_independente: Lista de variáveis independentes a serem avaliadas.
# - base: Conjunto de dados contendo as variáveis dependentes e independentes.
# Retorna: DataFrame contendo as combinações de variáveis selecionadas e seus respectivos AICs, 
# ordenados do menor para o maior AIC.
# criada por Mateus Rocha - time ASN.Rocks

def selecionar_aic_backward(var_dependente, var_independente, base):
    Y = base[var_dependente]
    
    preditoras_finais = []
    
    aic_final = []
    
    while True and var_independente != []:
        
        lista_aic = []
        lista_preditoras = []

        X_geral = sm.add_constant(base[var_independente])
        
        aic_geral = sm.OLS(Y,X_geral).fit().aic
    
        aic_final.append(aic_geral)
        
        preditoras_finais.append(base[var_independente].columns.to_list())
        
        for var in var_independente:
            
            lista_variaveis = var_independente.copy()
            lista_variaveis.remove(var)
            
            X = sm.add_constant(base[ lista_variaveis ])
            aic = sm.OLS(Y,X).fit().aic    
            
            lista_aic.append(aic)
            
            lista_preditoras.append(var)
            
        if(lista_aic[ np.argmin(lista_aic) ] < aic_geral ):
            var_independente.remove( lista_preditoras[ np.argmin(lista_aic) ] )
            
        else:
            break
    
    
    info_final = pd.DataFrame({ 'var': preditoras_finais, 'aic':aic_final }).sort_values(by = 'aic')
    return info_final

# Exemplo de uso
# colunas_aic_back = selecionar_aic_backward(var_dependente='bodyfat', var_independente=X.columns.to_list(), base=Bodyfat)
# colunas_aic_back

In [ ]:
# Esta função realiza uma seleção backward stepwise com base no critério de informação bayesiano (BIC).
# A cada passo, remove a variável que minimiza o BIC, desde que a remoção resulte em um BIC menor do que o modelo atual.
# Parâmetros:
# - var_dependente: Nome da variável dependente.
# - var_independente: Lista de variáveis independentes a serem avaliadas.
# - base: Conjunto de dados contendo as variáveis dependentes e independentes.
# Retorna: DataFrame contendo as combinações de variáveis selecionadas e seus respectivos BICs, 
# ordenados do menor para o maior BIC.
# criada por Mateus Rocha - time ASN.Rocks

def selecionar_bic_backward(var_dependente, var_independente, base):
    Y = base[var_dependente]
    
    preditoras_finais = []
    
    bic_final = []
    
    while True and var_independente != []:
        
        lista_bic = []
        lista_preditoras = []

        X_geral = sm.add_constant(base[var_independente])
        
        bic_geral = sm.OLS(Y,X_geral).fit().bic
    
        bic_final.append(bic_geral)
        
        preditoras_finais.append(base[var_independente].columns.to_list())
        
        for var in var_independente:
            
            lista_variaveis = var_independente.copy()
            lista_variaveis.remove(var)
            
            X = sm.add_constant(base[ lista_variaveis ])
            bic = sm.OLS(Y,X).fit().bic    
            
            lista_bic.append(bic)
            
            lista_preditoras.append(var)
            
        if(lista_bic[ np.argmin(lista_bic) ] < bic_geral ):
            var_independente.remove( lista_preditoras[ np.argmin(lista_bic) ] )
            
        else:
            break
    
    
    info_final = pd.DataFrame({ 'var': preditoras_finais, 'bic':bic_final }).sort_values(by = 'bic')
    return info_final

# Exemplo de uso
# colunas_bic_back = selecionar_bic_backward(var_dependente='bodyfat', var_independente=X.columns.to_list(), base=Bodyfat)
# colunas_bic_back

In [ ]:
# Esta função realiza a seleção stepwise de variáveis, usando os métodos forward e backward 
# com base em uma métrica específica (AIC, BIC ou p-valor).
# O processo consiste em primeiro aplicar a seleção forward com a métrica escolhida e, 
# em seguida, a backward, ajustando o modelo até que a diferença entre as métricas seja menor 
# que um valor de tolerância (epsilon).
# Parâmetros:
# - var_dependente: Nome da variável dependente.
# - var_independente: Lista de variáveis independentes a serem avaliadas.
# - base: Conjunto de dados contendo as variáveis dependentes e independentes.
# - metrica: A métrica a ser usada no processo de seleção (pode ser 'aic', 'bic', ou 'pvalor').
# - signif: Nível de significância usado para a seleção por p-valor (padrão 0.05).
# - epsilon: Diferença mínima aceitável entre as métricas forward e backward para parar o processo (padrão 0.0001).
# Retorna: DataFrame contendo as variáveis selecionadas e suas respectivas métricas (AIC, BIC ou p-valor), 
# dependendo da métrica usada.
# criada por Mateus Rocha - time ASN.Rocks

def stepwise( var_dependente , var_independente , base, metrica, signif = 0.05, epsilon = 0.0001):
    
    lista_var = var_independente
    
    metrica_forward = 0
    
    metrica_backward = 0
    
    while True:
    
        if(metrica == 'aic'):
            resultado = selecionar_aic_forward(var_dependente = var_dependente, var_independente = var_independente, base = base)

            if (len(resultado) == 1):
                return resultado
            
            resultado_final = selecionar_aic_backward(var_dependente = var_dependente, var_independente = resultado['var'].to_list()[0], base = base)

            if(len(resultado_final) == 1):
                return resultado_final

            metrica_forward = resultado['aic'].to_list()[0]

            metrica_backward = resultado_final['aic'].to_list()[0]


        elif(metrica == 'bic'):
            resultado = selecionar_bic_forward(var_dependente = var_dependente, var_independente = var_independente, base = base)

            if (len(resultado) == 1):
                return resultado

            resultado_final = selecionar_bic_backward(var_dependente = var_dependente, var_independente = resultado['var'].to_list()[0], base = base)

            if(len(resultado_final) == 1):
                return resultado_final

            metrica_forward = resultado['bic'].to_list()[0]

            metrica_backward = resultado_final['bic'].to_list()[0]

        elif(metrica == 'pvalor'):
            resultado = selecionar_pvalor_forward(var_dependente = var_dependente, var_independente = var_independente, base = base, signif = signif)

            if (len(resultado) == 1):
                return resultado

            resultado_final = selecionar_pvalor_backward(var_dependente = var_dependente, var_independente = resultado['var'].to_list(), base = base, signif = signif)

            if(len(resultado_final) == 1):
                return resultado_final

            return resultado_final

        if( abs(metrica_forward - metrica_backward) < epsilon ):
            break
        else:
            var_independente = set(resultado_final['var'].to_list() + lista_var)

# Exemplo de uso
# colunas_stepwise = stepwise(var_dependente='bodyfat', var_independente=X.columns.to_list(), base=Bodyfat, metrica='aic', signif=0.05)
# colunas_stepwise

In [ ]:
# Esta função realiza a seleção de variáveis usando os métodos forward, backward ou stepwise, com base em uma métrica escolhida (AIC, BIC ou p-valor).
# O usuário pode escolher o método de seleção (forward, backward ou both) e a métrica desejada para o critério de inclusão ou exclusão de variáveis.
# Parâmetros:
# - var_dependente: Nome da variável dependente.
# - var_independente: Lista de variáveis independentes a serem avaliadas.
# - base: Conjunto de dados contendo as variáveis dependentes e independentes.
# - metodo: Método de seleção ('forward', 'backward' ou 'both').
# - metrica: A métrica a ser utilizada para a seleção ('aic', 'bic' ou 'pvalor').
# - signif: Nível de significância usado para a seleção por p-valor (padrão 0.05).
# Retorna: Resultado da seleção de variáveis com base no método e métrica escolhidos.
# criada por Mateus Rocha - time ASN.Rocks

def step( var_dependente , var_independente , base, metodo, metrica, signif = 0.05):
    
    if( metodo == 'forward' and metrica == 'aic' ):
        resultado = selecionar_aic_forward(var_dependente = var_dependente, var_independente = var_independente, base = base)
    elif(metodo == 'forward' and metrica == 'bic' ):
        resultado = selecionar_bic_forward(var_dependente = var_dependente, var_independente = var_independente, base = base)
    elif(metodo == 'forward' and metrica == 'pvalor' ):
        resultado = selecionar_pvalor_forward(var_dependente = var_dependente, var_independente = var_independente, base = base, signif = signif)
    elif( metodo == 'backward' and metrica == 'aic' ):
        resultado = selecionar_aic_backward(var_dependente = var_dependente, var_independente = var_independente, base = base)
    elif(metodo == 'backward'and metrica == 'bic' ):
        resultado = selecionar_bic_backward(var_dependente = var_dependente, var_independente = var_independente, base = base)
    elif(metodo == 'backward' and metrica == 'pvalor' ):
        resultado = selecionar_pvalor_backward(var_dependente = var_dependente, var_independente = var_independente, base = base, signif = signif)
    elif(metodo == 'both'):
        resultado = stepwise( var_dependente = var_dependente , var_independente = var_independente , base = base, metrica = metrica, signif = signif)
        
    # Ajustar a exibição do pandas para não truncar as colunas e linhas longas
    pd.set_option('display.max_colwidth', None)  # Não cortar as colunas
    pd.set_option('display.max_rows', None)  # Mostrar todas as linhas
    
    return resultado
    
# Exemplo de uso
# colunas_step = step(var_dependente='bodyfat', var_independente=X.columns.to_list(), base=Bodyfat, metodo='both', metrica='aic', signif=0.05)
# colunas_step

## <center>1. Definição do problema de negócio

### <center>Contexto

Para termos uma medida precisa da gordura corporal do nosso corpo, o processo é custoso, inconveniente e cansativo. Sendo assim, é desejável que existam métodos fáceis de estimar a gordura corporal que não sejam inconvenientes, nem tão custosos.
Com os dados do arquivo Bodyfat.csv, criaremos um modelo para identificar a melhor forma de prever o valor do IMC utilizando variáveis independentes, validaremos o modelo e verificaremos se não tem multicolinearidade.

## <center>2. Entendimento e aquisição de dados

### <center>Importando arquivo de dados

In [ ]:
df = pd.read_csv("Bodyfat.csv")
df = df.select_dtypes(include=['number']) # Filtrar apenas colunas numéricas

### <center>Dicionário de dados

| Variável  | Descrição | Unidade de Medida | Tipo | Comentário |
|-----------|-----------|------------------|------|------------|
| Density   | Densidade corporal, usada para calcular a gordura corporal | - | Quantitativa contínua | Não será utilizada na análise |
| bodyfat   | Percentual de gordura corporal, a variável alvo | % | Quantitativa contínua | Variável dependente (target). Validar outlier com a área de negócio |
| Age       | Idade do indivíduo | Anos | Quantitativa discreta | - |
| Weight    | Peso do indivíduo | Libras | Quantitativa contínua | - |
| Height    | Altura do indivíduo | Polegadas | Quantitativa contínua | Tratar outlier de altura |
| Neck      | Circunferência do pescoço | Polegadas | Quantitativa contínua | - |
| Chest     | Circunferência do peito | Polegadas | Quantitativa contínua | - |
| Abdomen   | Circunferência abdominal | Polegadas | Quantitativa contínua | - |
| Hip       | Circunferência do quadril | Polegadas | Quantitativa contínua | - |
| Thigh     | Circunferência da coxa | Polegadas | Quantitativa contínua | - |
| Knee      | Circunferência do joelho | Polegadas | Quantitativa contínua | - |
| Ankle     | Circunferência do tornozelo | Polegadas | Quantitativa contínua | - |
| Biceps    | Circunferência do bíceps em repouso | Polegadas | Quantitativa contínua | - |
| Forearm   | Circunferência do antebraço | Polegadas | Quantitativa contínua | - |
| Wrist     | Circunferência do punho | Polegadas | Quantitativa contínua | - |

### <center>Conhecendo minhas variáveis

#### <code style="color:green">Análise Univariada</code>

Antes de realizar qualquer modelagem, devemos sempre analisar nossas variáveis, em especial a variável dependente.

Podemos fazer uso de ferramentas como:

- Análises Descritivas;
- Gráficos boxplot;
- Gráficos histograma;
- Gráficos de dispersão.

A ideia é analisar do que se tratam os dados e se possuem algo estranho.

- Observar se a mediana e média estão próximas;
- Verificar se existem valores estranhos de mínimo e máximo, por exemplo, valores negativos no mínimo e valores = 0 no máx;
- Observar o movimento dos dados analisando os quartis;
- Criar coluna no dicionário de dados para anotar observações sobre cada variável para alinhar com a área de negócio.

In [ ]:
# Verificando as colunas e primeiras linhas da base de dados
df.head()

In [ ]:
# Transformando unidade de medida das variáveis para facilitar interpretação

# Transformando a variável weight de libras para quilos
df['Weight'] = df['Weight'] * 0.453592

# Transformando as variáveis em polegadas para centímetros
df['Height'] = df['Height'] * 2.54
df['Neck'] = df['Neck'] * 2.54
df['Chest'] = df['Chest'] * 2.54
df['Abdomen'] = df['Abdomen'] * 2.54
df['Hip'] = df['Hip'] * 2.54
df['Thigh'] = df['Thigh'] * 2.54
df['Knee'] = df['Knee'] * 2.54
df['Ankle'] = df['Ankle'] * 2.54
df['Biceps'] = df['Biceps'] * 2.54
df['Forearm'] = df['Forearm'] * 2.54
df['Wrist'] = df['Wrist'] * 2.54

In [ ]:
# Verificando valores faltantes e tipos de dados de cada campo
df.info()

In [ ]:
# Verificando principais valores estatísticos das variáveis em questão. Atenção especial à variável dependente (y)
df.describe().T

In [ ]:
# Gerando boxplot, histograma e violino de cada variável
def plot_histogram_boxplot(df):
    
    # Gera histogramas e boxplots para todas as colunas numéricas de um DataFrame.

    num_cols = df.select_dtypes(include=['number']).columns  # Filtra apenas colunas numéricas
    
    for coluna in num_cols:
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))  # Criar subplots lado a lado

        # Histograma com densidade
        sns.histplot(df[coluna], kde=True, bins=30, color="#39568C", ax=axes[0]) # kde=True adiciona a curva de densidade
        axes[0].set_title(f"Histograma de {coluna}")
        axes[0].set_ylabel("Densidade")
        axes[0].axvline(df[coluna].mean(), color='red', linestyle='dashed', linewidth=2, label="Mean")  # Linha da média
        axes[0].legend() 

        # Boxplot
        sns.boxplot(y=df[coluna], color="#F8766D", ax=axes[1])
        axes[1].set_title(f"Boxplot de {coluna}")

        # Gráfico de violino
        sns.violinplot(y=df[coluna], ax=axes[2])
        axes[2].set_title(f"Gráfico de Violino de {coluna}")

        plt.tight_layout()
        plt.show()

# Exemplo de uso:
plot_histogram_boxplot(df)

## <center>3. Preparação dos dados

#### <code style="color:green">Pré-processamento e Limpeza dos dados</code>

- Verificar observações sobre a análise univariada com a área de negócio e realizar tratamento dos dados
- Criar ABT
- Realizar transformação de dados quando necessário

In [ ]:
# tirando uma variavel que não será usada (contexto negocio)
df = df.drop(["Density"], axis=1)
df

In [ ]:
# Atribuir o valor da média da altura de todas as observações (adultos) ao outlier de Height ( = 74.93)

df['Height'] = df['Height'].replace(74.93, df['Height'].mean())

df.describe().T


## <center>4. Análise exploratória dos dados

#### <code style="color:green">Etapas da EDA</code>

- Análise bivariada
- Avaliar e tratar multicolinearidade
- Fazer descritiva com foco no modelo
- Validar pressupostos

#### <code style="color:green">Análise Bivariada</code>

- Cruzar variáveis e identificar correlação de pearson
- Observar relação entre cada variável independente e a variável dependente para identificar correlação visualmente (vontade de traçar uma reta angular?)
- Analisar matriz de correlação (acima de 0,7 indica alta correlação)
- Escolher quais variáveis manter no modelo

In [ ]:
# Gerando gráficos de dispersão entre todas as variáveis

# Ajustar o tamanho da figura
plt.figure(figsize=(12, 10))  # Ajuste o tamanho conforme necessário

# Verificar linearidade (gráficos de dispersão)
sns.pairplot(df, corner=True)

# Mostrar o gráfico
plt.show()

In [ ]:
# Heatmap para cálculo de correlação com valores de correlação

# corr_matrix = pd.concat([y, X], axis=1).corr()
corr_matrix = df.corr()

# Criar a máscara para a parte superior da matriz, evitando redundância já que a matriz é simétrica
mask = np.triu(np.ones_like(corr_matrix, dtype=bool)) 

# Ajustar o tamanho da figura
plt.figure(figsize=(12, 10))

# Criar o heatmap
sns.heatmap(corr_matrix, annot=True, fmt='.1f', center=0, vmax=1, vmin=-1, cmap="RdBu_r", annot_kws={"size": 10}, mask=mask)
plt.show()


<code style="color:purple">Observações</code>

- Pode-se observar que as variáveis Ankle, Age e Height, no gráfico de dispersão, não apresentam um comportamente linear. Vamos retirá-las do df para rodar o primeiro modelo.

- Também observa-se que a variável Weight possui correlação forte com Hip, Abdomen, Chest, Thigh e Knee. Isso pode causar multicolinearidade no modelo.

<code style="color:red">Dúvida</code>

   - Ao observar que a variável Weight possui correlação forte com várias outras variáveis, como testar qual podemos retirar para gerar o melhor modelo?

## <center>5. Modelagem

### <center>Criando o modelo de regressão linear múltipla

In [ ]:
# Definir a variável dependente (y) e as independentes (X)

# Criando o objeto y com a variável dependente
y = df["bodyfat"]
# Criando a matriz X sem a variável dependente
X = df.drop(columns=["bodyfat"])  # Usando todas as variáveis independentes numéricas

# Adicionando uma coluna com todos os valores = 1 para que o B0 seja calculado
X = sm.add_constant(X)

# Ajustando o modelo
modelo_full = sm.OLS(y, X).fit()
# Resumo do modelo
print(modelo_full.summary())
# Gerando a predição através deste modelo
chute_modelo_full = modelo_full.predict(X)

"""
Existe a opção de utilizar o Scikit-Learn para predições e deploy, mas é menos detalhado para análise estatística. Não precisa adicionar constante. 

Para utilizar este método, separaríamos as observações em test e treino
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

modelo = LinearRegression()
modelo.fit(X_train, y_train)
"""

<code style="color:purple">Resultados do modelo com todas as variáveis independentes</code>

De acordo com os resultados obtidos na regressão utilizando todas as variáveis independentes, observa-se que:

- **P>|t|**  A maior parte das variáveis independentes deste modelo são não significativas.
- **Test F (Prob (F-statistic))**:	9.75e-64, indica que pelo menos 1 variável independente é significativa.
- **Cond. No.** = 4.37e+04 nos dá um indício do motivo deste resultado, já que por ser muito grande, indica alta multicolinearidade entre as variáveis.
- **R2** = 0.749: O modelo explica 74.9% da variação do percentual de gordura corporal (bodyfat), o que indica um bom ajuste.
- **𝑅2 ajustado** = 0.735 → Leva em conta o número de variáveis no modelo e é um pouco menor, sugerindo que algumas variáveis podem não estar contribuindo significativamente.
**Estatística F** = 54.50 → Testa se pelo menos uma variável independente tem um coeficiente diferente de zero.
**p-valor do F-statistic**: 9.75e-64 → Extremamente pequeno, indicando que o modelo é estatisticamente significativo.

<code style="color:purple">Comentários</code>

Antes de utilizar um método de seleção de variáveis, vamos retirar algumas variáveis do df que foram identificadas com baixa correlação com a variável dependente. 

In [ ]:
# Criando uma nova matriz X sem as variáveis Height, Age e Ankle
X_manual = df.drop(columns=["bodyfat", "Height", "Age", "Ankle"])  # Usando todas as variáveis independentes numéricas, exceto Height, Age e Ankle

# Aqui lembre-se do B0. Basicamente vamos adicionar uma coluna de 1s para que o B0 seja calculado.
X_manual = sm.add_constant(X_manual)
# Ajustando o modelo
manual = sm.OLS(y, X_manual).fit()
# Resumo do modelo
print(manual.summary()) 
# gerando a predição aravés desse modelo
pred_manual = manual.predict(X_manual)


### <code style="color:green">Seleção de Variáveis</code>

Seleção de variáveis é uma etapa que objetiva selecionar apenas as variáveis úteis para o modelo.

#### <center>Métodos:

1: **Lasso (Regularização L1)**: Usa uma penalização (L1) que força coeficientes irrelevantes a zero, eliminando variáveis automaticamente.
- Melhor para: Modelos lineares, muitas variáveis
- Vantagens: Fácil de interpretar, evita overfitting
- Desvantagens: Pode descartar variáveis importantes
- Popularidade: 4

2: **Random Forest/XGBoost**: Calculam a importância de cada variável no modelo.
- Melhor para: Qualquer modelo
- Vantagens: Considera interações e relações não lineares
- Desvantagens: Difícil de interpretar
- Popularidade: 5

3: **RFE (Seleção Recursiva)**: Remove iterativamente a variável menos importante até encontrar o melhor conjunto.
- Melhor para: Modelos específicos (Linear, SVM)
- Vantagens: Boa seleção para modelos pequenos
- Desvantagens: Computacionalmente caro
- Popularidade: 3

4: **Stepwise Selection**: Testa variáveis iterativamente, adicionando (forward), removendo (backward) ou integrando o backward e forward juntos com base no p-valor ou AIC/BIC
- Melhor para: Modelos estatísticos
- Vantagens: Interpretável
- Desvantagens: Propenso a overfitting, lento
- Popularidade: 2

5: **Mutual Information**: Mede a dependência entre variáveis (entropia).
- Melhor para: Dados categóricos
- Vantagens: Lida em com não-linearidade
- Desvantagens: Difícil de interpretar
- Popularidade: 2

#### <center>Forward AIC

In [ ]:
# Realizando a seleção de variáveis pelo método forward com base no AIC, utilizando as funções criadas no início do documento


# Criando o objeto X com as variáveis independentes, ou seja, tirando o que foi feito antes e a variável resposta

X = df.drop(["bodyfat"],axis = 1)

# Criando o objeto y com a variável dependente
y = df["bodyfat"]

colunas_forw = step(var_dependente = 'bodyfat', var_independente = X.columns.to_list(), 
                    base = df, metodo = 'forward', metrica = 'aic')
colunas_forw

#### <code style="color:purple">Observações: </code>

- Note que as melhores variáveis estão na primeira linha.
- Tanto no AIC quanto no BIC, quanto menor for o valor, melhor o modelo será
- Agora vamos pegar essas colunas e criar o modelo

In [ ]:
# colocando em uma lista o nome de todas as variaveis do modelo com menor aic
X_forw = df[colunas_forw['var'].to_list()[0]]
# Aqui lembre-se do B0. Basicamente vamos adicionar uma coluna de 1s para que o B0 seja calculado.
X_forw = sm.add_constant(X_forw)
# Ajustando o modelo
forw = sm.OLS(y, X_forw).fit()
# Resumo do modelo
print(forw.summary()) 
# gerando a predição aravés desse modelo
pred_forw = forw.predict(X_forw)

#### <code style="color:purple">Observações: </code>

Por que alguns p-valores são não significativos?

#### <center>Backward AIC

In [ ]:
# regressao por backward com a regra de AIC
colunas_backw = step(var_dependente = 'bodyfat', var_independente = X.columns.to_list(), base = df, metodo = 'backward' ,metrica = 'aic')
colunas_backw

In [ ]:
# colocando em uma lista o nome de todas as variaveis do modelo com menor aic
X_backw = df[colunas_backw['var'].to_list()[0]] 
# Aqui lembre-se do B0. Basicamente vamos adicionar uma coluna de 1s para que o B0 seja calculado.
X_backw = sm.add_constant(X_backw)
# Ajustando o modelo
backw = sm.OLS(y, X_backw).fit()
# Resumo do modelo
print(backw.summary()) 
# gerando a predição aravés desse modelo
pred_backw = backw.predict(X_backw)

#### <center>Stepwise AIC

In [ ]:
# regressao por stepwise com a regra de AIC
colunas_stepw = step(var_dependente = 'bodyfat', var_independente = X.columns.to_list(), base = df, metodo = 'both' ,metrica = 'aic')
colunas_stepw

In [ ]:
# colocando em uma lista o nome de todas as variaveis do modelo com menor aic
X_stepw = df [ colunas_stepw['var'].to_list()[0] ] 
# Aqui lembre-se do B0. Basicamente vamos adicionar uma coluna de 1s para que o B0 seja calculado.
X_stepw = sm.add_constant(X_stepw)
# Ajustando o modelo
stepw = sm.OLS(y, X_stepw).fit()
# Resumo do modelo
print(stepw.summary()) 
# gerando a predição aravés desse modelo
pred_stepw = stepw.predict(X_stepw)

In [ ]:
X

In [ ]:
# regressao por stepwise com a regra de p-valor


colunas_stepw_p = step(var_dependente = 'bodyfat', var_independente = X.columns.to_list(), base = df, metodo = 'both' ,metrica = 'pvalor')
colunas_stepw_p


In [ ]:
# colocando em uma lista o nome de todas as variaveis do modelo com p-valor
X_stepw_p = df[colunas_stepw_p['var'].to_list()] 
# Aqui lembre-se do B0. Basicamente vamos adicionar uma coluna de 1s para que o B0 seja calculado.
X_stepw_p = sm.add_constant(X_stepw_p)
# Ajustando o modelo
stepw_p = sm.OLS(y, X_stepw_p).fit()
# Resumo do modelo
print(stepw_p.summary()) 
# gerando a predição aravés desse modelo
pred_stepw_p = stepw_p.predict(X_stepw_p)

### <center>Verificando a qualidade dos modelos

### <code style="color:green">Interpretando os Resultados - Métricas de Qualidade de Ajuste</code>

- **$R^{2}$**: Mede a proporção da variação de y explicada pelo modelo. Quanto mais próximo de 1, melhor o ajuste.
- **$R^{2}$ ajustado**: Considera o número de variáveis no modelo, penalizando modelos com variáveis explicativas irrelevantes. Se muito menor que $R^{2}$, pode indicar que algumas variáveis não contribuem significativamente. É útil para comparar modelos com diferentes quantidades de variáveis.
- **Estatística F**: Testa a significância global do modelo, ou seja, se pelo menos uma variável independente tem um coeficiente diferente de zero. F alto, melhor. É a estatística do teste que só poderá ser interpretada se comparada a estatística F tabelada com o nível de significância de interesse.
- **Prob (F-statistic)**: É a probabilidade da estatística F. Se (Prob (F-statistic)) < 0,05 (nível de significância), rejeita-se H0, ou seja, pelo menos uma variável independente é significativa.
\begin{align*}
H_0: & \quad \beta_1 = \beta_2 = \cdots = \beta_k = 0 \\
H_1: & \quad \text{Ao menos um  é } \beta_i \neq 0
\end{align*}
- **Log-Likelihood**: Mede a qualidade do ajuste (log-verossimilhança); Quanto mais positivos, melhor o modelo ajusta os dados.
- **AIC e BIC**: Critérios para comparar modelos; valores menores indicam melhor ajuste com menos complexidade. BIC penaliza mais modelos com muitas variáveis.
- **Coeficientes (coef)**: Representam o impacto de cada variável independente sobre y. Valores positivos indicam relação direta, e negativos indicam relação inversa. É o valor da constante no modelo de regressão. Representa o valor esperado da variável dependente quando todas as variáveis independentes são zero. Não tem interpretação.
- **Erro padrão (std err):** Mede a incerteza na estimativa do coeficiente. Indica a precisão do coeficiente estimado. Um erro padrão menor indica uma estimativa mais precisa (ou seja, em diferentes amostras os resultados seriam os mesmos).
- **Valor t (t)**: É o valor da estatística de teste para a significância dos coeficientes.
- **P>|t|**: Representa o resultado do teste t para cada variável. Quando P>|t| > 0,05, não rejeitamos H0 (B=0), indicando que a variável pode não ser significativa para prever y. p>|t| < 0.05 indica que a variável tem efeito significativo no modelo.
\begin{align*}
H_0: & \quad \beta_i = 0 \\
H_1: & \quad \beta_i \neq 0
\end{align*}
- **Intervalo de confiança [0.025, 0.975]**: Indica a faixa na qual o coeficiente pode variar com 95% de confiança. Se o intervalo inclui 0, a variável pode não ser significativa.
- **condition number**: Se muito grande, indica multicolinearidade.


#### Como interpretar o modelo:
- Se **muitas variáveis não são significativas** (P>|t| > 0,05), pode ser necessário refazer a seleção de variáveis.
- Se **$R^{2}$ for baixo**, o modelo pode não explicar bem a variabilidade da variável dependente. Mas se for alto demais, pode indicar sobreajuste (overfitting).
- Se **$R^{2}$ ajustado for muito menor que $R^{2}$**, o modelo pode estar incluindo variáveis irrelevantes.
- Se **AIC/BIC forem muito altos**, pode haver um modelo mais simples com melhor desempenho.
- Se coeficientes com p-valores baixos são mais confiáveis na previsão da variável dependente.
- Se muitas variáveis têm p-valores altos, pode ser necessário simplificar o modelo.


#### Calculando MAE e RMSE

- **RMSE**: Valores mais próximos de 0 indicam um bom RMSE no geral, mas se o RMSE do modelo for melhor que o RMSE da média, então podemos considerar que é um bom modelo.
- **MAE**: O MAE também mede o erro médio das previsões, mas sem elevar ao quadrado as diferenças, sendo mais interpretável e menos sensível a outliers do que o RMSE.
Fórmula do RMSE:
\begin{equation} RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2} \end{equation}

onde:

$n$ é o número de observações.
$y_i$ é o valor observado.
$\hat{y}_i$ é o valor predito.
Fórmula do MAE:
\begin{equation} MAE = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i| \end{equation}

onde:

$n$ é o número de observações.
$y_i$ é o valor observado.
$\hat{y}_i$ é o valor predito.

**Interpretação**:

MAE mede o erro médio absoluto das previsões, penalizando todas as diferenças de forma linear.
RMSE penaliza erros maiores de forma quadrática, sendo mais sensível a outliers.
Comparação: Se MAE e RMSE são próximos, os erros estão distribuídos de maneira uniforme. Se RMSE for muito maior que o MAE, significa que há alguns erros muito grandes (outliers influentes).

In [ ]:
# Criar um DataFrame para armazenar os resultados de cada modelo (AIC, BIC, RMSE, MAE e R²)

resultados = pd.DataFrame({
    'Modelo': ['Modelo Completo', 'Modelo Manual', 'Forward Stepwise', 'Backward Stepwise', 'Stepwise AIC', 'Stepwise P-Valor'],
    'AIC': [modelo_full.aic, manual.aic, forw.aic, backw.aic, stepw.aic, stepw_p.aic],
    'BIC': [modelo_full.bic, manual.bic, forw.bic, backw.bic, stepw.bic, stepw_p.bic],
    'RMSE': [np.sqrt(mean_squared_error(y, chute_modelo_full)), 
             np.sqrt(mean_squared_error(y, pred_manual)), 
             np.sqrt(mean_squared_error(y, pred_forw)), 
             np.sqrt(mean_squared_error(y, pred_backw)), 
             np.sqrt(mean_squared_error(y, pred_stepw)), 
             np.sqrt(mean_squared_error(y, pred_stepw_p))],
    'R²': [r2_score(y, chute_modelo_full), 
           r2_score(y, pred_manual), 
           r2_score(y, pred_forw), 
           r2_score(y, pred_backw), 
           r2_score(y, pred_stepw), 
           r2_score(y, pred_stepw_p)],
    'MAE': [mean_absolute_error(y, chute_modelo_full), 
            mean_absolute_error(y, pred_manual), 
            mean_absolute_error(y, pred_forw), 
            mean_absolute_error(y, pred_backw), 
            mean_absolute_error(y, pred_stepw), 
            mean_absolute_error(y, pred_stepw_p)]
})

df_resultados = pd.DataFrame(resultados)
# Exibir os resultados formatados como tabela
df_resultados

### <code style="color:green">Comparando modelos</code>

- **AIC, BIC, RMSE E MAE**: Quanto menor, melhor
- **${R^2}$**: Quanto maior, melhor

In [ ]:
# Como os modelos ficaram os mesmos (backward pelo AIC, forward pelo AIC e setpwise pelo AIC)
tentativas = pd.DataFrame({
'chute_modelo_full': chute_modelo_full,
'chute_modelo_step_aic': pred_stepw,
'chute_modelo_step_pvalor':pred_stepw_p,
'chute_modelo_manual': pred_manual,
})
tentativas

In [ ]:
# definindo o tamanho da área do gráfico
plt.figure(figsize=(10, 6))
# adicionando a primeira camada que são nossos dados reais em azul
plt.scatter(df["bodyfat"], df["bodyfat"], color='blue', label='real')
# adicionando a camada do chute da média em vermelho
df["chute_media"] = df["bodyfat"].mean()
plt.scatter(df["chute_media"], df["bodyfat"], color='red', label='chute_media')
# adicionando a camada das nossas predições do modelo completo (sem nenhuma seleção) em verde
plt.scatter(tentativas["chute_modelo_full"], df["bodyfat"], color='green', label='chute_modelo_full')
# adicionando a camada das nossas predições do modelo stepwise pelo aic em laranja
plt.scatter(tentativas["chute_modelo_step_aic"], df["bodyfat"], color='orange', label='chute_modelo_step_aic')
# adicionando a camada das nossas predições do modelo stepwise por p-valor em pink
plt.scatter(tentativas["chute_modelo_step_pvalor"], df["bodyfat"], color='pink', label='chute_modelo_step_pvalor')
# adicionando a camada das nossas predições do modelo manual em roxo
plt.scatter(tentativas["chute_modelo_manual"], df["bodyfat"], color='purple', label='chute_modelo_manual')

plt.xlabel('Nosso Desenho')
plt.ylabel('Arte Real')
plt.legend()
plt.show()

<code style="color:purple">Comentários:</code>

Vamos seguir com a análise do modelo stepw_p

#### <code style="color:green">Comparar efeitos e variações</code>

- Analisar os nossos coeficientes observando o intervalo de confiança (apesar que já sai no summary)
- Vamos gerar os gráficos dos intervalos para facilitar a visualização

In [ ]:
# pegando o intervalo de confiança dos coeficientes menos o B0
intervalo_confianca_forward = stepw_p.conf_int(alpha=0.05)[1:] 
intervalo_confianca_forward

In [ ]:
# criando o gráfico para o intervalo de confiança
fig, ax = plt.subplots() 

ax.errorbar(stepw_p.params.drop(['const']), # pegando os valores dos coeficientes e retirando a constante 
            stepw_p.params.drop(['const']).index,  # pegando os nomes dos coeficientes e retirando a constante
            xerr=[stepw_p.params.drop(['const']) - intervalo_confianca_forward[0], # criando as barras dos intervalos (limite inferior) 
                  intervalo_confianca_forward[1] - stepw_p.params.drop(['const'])], # criando as barras dos intervalos (limite superior)
            fmt='o', capsize=5)

ax.set_xlabel('Coeficientes') # adicionando o rótulo do eixo x
ax.set_ylabel('Variáveis') # adicionando o rótulo do eixo y
ax.axvline(x=0,linestyle='--', color = "#440154FF") # criando uma linha vertical no x = 0
plt.xticks(rotation=45) # adicionando uma rotação de 45 graus nos valores de x
plt.tight_layout()
plt.show()

<code style="color:purple">Comentários:</code>

- Quando algum intervalo corta a linha do 0 significa que o coeficiente não foi significativo.
- Note que todas, neste caso, foram significativas.

### <center>Validando o modelo

#### <code style="color:green">Análise de resíduos</code>

O resíduo nada mais é que o erro do nosso modelo, ou seja o quanto ele está errando ao tentar prever a variável dependente. Nesse caso, devemos analisar um conjunto de características para considerar o nosso modelo como um modelo robusto:
 
- Homogeneidade de variâncias: Vamos analisar o gráfico das predições sobre os resíduos. Se tiver algum tipo de padrão então as variâncias dos nossos resíduos são heterocedásticas, isto é, não homogêneas.

- Normalidade: Os resíduos precisam seguir uma distribuição normal com média igual a 0. Para isso vamos usar o QQ-Plot e teste estatístico

- Independência: Os resíduos não podem apresentar nenhum padrão conforme a variação de ychapeu

#### <code style="color:green">Qualidade dos Resíduos</code>

- **Omnibus**: O teste Omnibus avalia se os resíduos seguem uma distribuição normal, considerando a assimetria e a curtose dos resíduos. Um valor de p grande sugere que os resíduos podem ser normais. Este valor representa a estatística do teste.

- **Prob(Omnibus)**: Valor-p associado ao teste Omnibus, indicando a probabilidade de observar o resultado dado se a hipótese nula de distribuição normal for verdadeira. Idealmente, esperamos que os resíduos sejam normais para garantir que os pressupostos do modelo estejam corretos.

\begin{align*}
H_{0}: & \quad \text{Os resíduos são normais} \\
H_{1}: & \quad \text{Os resíduos não são normais}
\end{align*}

- **Skew**: Mede a assimetria da distribuição dos dados. Valores negativos de Skew indicam uma cauda mais longa à esquerda (distribuição assimétrica negativa), enquanto valores positivos indicam uma cauda mais longa à direita (distribuição assimétrica positiva).

- **Kurtosis**: Mede se a distribuição dos dados tem caudas mais pesadas ou mais leves em relação à distribuição normal. Valores de curtose maiores que 3 indicam que os dados têm caudas mais pesadas ou uma distribuição mais pontuda do que a normal.

- **Durbin-Watson**: Testa a autocorrelação nos resíduos de uma análise de regressão. O ideal é que os resíduos não sejam correlacionados entre si. Valores de Durbin-Watson próximos de 2 sugerem que não há autocorrelação nos resíduos. Valores próximos de 0 indicam autocorrelação positiva (os resíduos consecutivos estão correlacionados), enquanto valores próximos de 4 indicam autocorrelação negativa.

\begin{align*}
H_0: & \quad \text{Não há autocorrelação nos resíduos} \\
H_1: & \quad \text{Há autocorrelação nos resíduos}
\end{align*}

- **Jarque-Bera (JB)**: O teste de Jarque-Bera verifica se os resíduos têm assimetria e curtose compatíveis com uma distribuição normal, baseando-se em uma combinação dessas duas métricas.

- **Prob(JB)**: Valor-p para o teste Jarque-Bera, avaliando a probabilidade de observar tais estatísticas se a assimetria e a curtose corresponderem a uma distribuição normal.

\begin{align*}
H_0: & \quad \text{Os dados possuem assimetria e curtose que correspondem a uma distribuição normal} \\
H_1: & \quad \text{Os dados não possuem assimetria e curtose que correspondem a uma distribuição normal}
\end{align*}

- **Cond. No**: O Número de Condição avalia a multicolinearidade, ou seja, a correlação entre as variáveis independentes. Valores muito altos (geralmente acima de 30) indicam forte multicolinearidade, o que pode comprometer a estabilidade dos coeficientes estimados no modelo.


In [ ]:
# criando gráfico de resíduos (resíduo x predito)
plt.figure(figsize=(10, 6))
plt.scatter(tentativas["chute_modelo_step_pvalor"],stepw_p.resid, color='black')
plt.axhline(y=0, color = 'black')
plt.xlabel('Valores Ajustados')
plt.ylabel('Resíduos')
plt.show()

#### <code style="color:purple">Observações:</code>

- Note que os resíduos não apresentam nenhum padrão, ou seja, demonstram independência.

#### <code style="color:green">Teste de Normalidade de Shapiro + QQ-plot + Histograma</code>

Gráficos são um pouco subjetivos. Dependendo do problema não dá para enxergar algo ou tirar uma conclusão. 
Testes estatísticos são mais confiáveis, portanto é melhor testar a normalidade e a homogeneidade de variâncias.

Normalidade (Shapiro-Wilk):

- $H_{0}:$ Os resíduos são normais
- $H_{1}:$ Os resíduos não são normais

In [ ]:
# ele gera a estatística do teste e o p-valor
shapiro(stepw_p.resid) 

In [ ]:
# qqplot 
stats.probplot(stepw_p.resid, dist="norm",plot=pylab)
pylab.plot()

In [ ]:
# visualizando o histograma
stepw_p.resid.plot(kind='hist', bins = 50)
plt.show()

#### <code style="color:purple">Observações:</code>

- Note que não rejeitamos $H_{0}$, logo os resíduos podem ser considerados como normais.
- Para que sejam considerados normais, os resíduos precisam seguir a linha em vermelho no qq-plot, o que pode indicar, novamente, normalidade.
- Pelo histograma, vemos um gráfico bem próximo da normalidade

#### <code style="color:green">Teste de Homogeneidade de Variância</code>

Validando o pressuposto de Homogeneidade de Variâncias através do teste de Breusch-Pagan:

- $H_{0}:$ Os resíduos possuem homogeneidade nas variâncias
- $H_{1}:$ Os resíduos não possuem homogeneidade nas variâncias

In [ ]:
# Realizando o teste de Breusch-Pagan para verificar heterocedasticidade
# Passamos os resíduos do modelo (stepw_p.resid) e as variáveis explicativas (stepw_p.model.exog)
teste_bp = het_breuschpagan(stepw_p.resid, stepw_p.model.exog)
# Definindo os rótulos para os resultados do teste
rotulos = ['LM Statistic', 'LM-Test p-value', 'F-Statistic', 'F-Test p-value']
# Usando zip para associar cada rótulo ao seu respectivo resultado do teste e convertendo em um dicionário
resultado_bp = dict(zip(rotulos, teste_bp))
# Exibindo o p-valor do LM-Test
print(resultado_bp['LM-Test p-value'])

#### <code style="color:purple">Observações:</code>

- Não rejeitamos $H_{0}$, logo as variâncias dos resíduos podem ser consideradas homogêneas.
- Sendo assim, esse modelo passou nos 4 pressupostos!

#### <code style="color:green">Multicolinearidade</code>

A multicolinearidade é um problema quando estamos tentando criar nosso modelo. Ela ocorre quando temos um conjunto de variáveis INDEPENDENTES que são altamente correlacionadas umas com as outras, o que gera um modelo com pouca confiabilidade. 

Vou analisar direto o VIF do modelo stepw_p:

In [ ]:
# criando um dataframe vazio
vif = pd.DataFrame() 
# adicionando as colunas que você quer analisar (deixe a coluna const!)
vif["Variáveis"] =X_stepw_p.columns 
# para cada coluna, calcule o VIF
vif["VIF"] = [variance_inflation_factor(X_stepw_p.values, i) for i in range(len(X_stepw_p.columns))] 
vif

#### <code style="color:purple">Observações:</code>

- Normalmente, utiliza-se VIF entre 3 ou 5, ou seja, este modelo, por mais que tenha passado na análise de resíduo está com o dilema da multicolinearidade.

- Uma variável pode influenciar no VIF da outra, por isso sempre começar observando a matriz de correlação a fim de selecionar as variáveis que fazem mais sentido serem mantidas. Diminuindo assim a chance de ter VIF alto.

In [ ]:
# vamos pegar da primeira até  a 14 coluna
correlacao = df[df.columns[np.arange(0,13)]].corr(method='pearson')

# Visualiza a matriz de correlação
plt.figure(figsize=(10, 8))
sns.heatmap(correlacao, annot=True, cmap='coolwarm', fmt='.2f', vmin=-1, vmax=1)
plt.title('Matriz de Correlação')
plt.show()

"""
# Heatmap para cálculo de correlação

# corr_matrix = pd.concat([y, X], axis=1).corr()
corr_matrix = df.corr()

# Criar a máscara para a parte superior da matriz, evitando redundância já que a matriz é simétrica
mask = np.triu(np.ones_like(corr_matrix, dtype=bool)) 

# Ajustar o tamanho da figura
plt.figure(figsize=(12, 10))

# Criar o heatmap
sns.heatmap(corr_matrix, annot=True, fmt='.2f', center=0, vmax=1, vmin=-1, cmap="RdBu_r", annot_kws={"size": 10}, mask=mask)
plt.show()
"""